# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [49]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [50]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("mycities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bilibino,68.0546,166.4372,4.23,78,15,1.63,Bilibino,1688652943
1,1,ta`u,-14.2336,-169.5144,27.53,75,95,8.20,Ta`ū,1688652944
2,2,bethel,41.3712,-73.4140,31.20,65,0,2.06,Bethel,1688652944
3,3,reggane,26.7158,0.1714,48.30,4,93,5.03,Reggane,1688652933
4,4,tommot,58.9564,126.2925,19.55,95,100,1.06,Tommot,1688652944


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [51]:
%%capture --no-display

# Configure the map plot
humidity_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = 'Humidity',
    tiles = "OSM",
    hover_cols=['Humidity'],
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "City",
    ylabel = "Latitude",
    xlabel = "Longitude",
)

humidity_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [54]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Humidity'].between(40, 70, inclusive='both')) &
    (city_data_df['Cloudiness'] < 5) &
    (city_data_df['Wind Speed'] < 5) &
    (city_data_df['Max Temp'].between(25, 30, inclusive='both'))
].copy()

# Drop any rows with null values
ideal_cities_df.dropna(inplace=True)

# Display sample data
ideal_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,42,remire-montjoly,4.9167,-52.2667,29.02,70,0,1.54,Rémire-Montjoly,1688652760
83,83,jamestown,42.0970,-79.2353,28.77,69,0,2.06,Jamestown,1688652966
102,102,cabo san lucas,22.8909,-109.9124,28.67,66,0,4.49,Cabo San Lucas,1688652972
130,130,la passe,45.5549,-0.8967,27.13,47,0,3.58,La Passe,1688652982
182,182,mao,39.8885,4.2658,29.31,45,0,4.63,Mahon,1688653008


### Step 3: Create a new DataFrame called `hotel_df`.

In [55]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.filter(
    ['City', 'Country', 'Lat', 'Lng', 'Humidity' ]
).copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
42,remire-montjoly,Rémire-Montjoly,4.9167,-52.2667,70,
83,jamestown,Jamestown,42.0970,-79.2353,69,
102,cabo san lucas,Cabo San Lucas,22.8909,-109.9124,66,
130,la passe,La Passe,45.5549,-0.8967,47,
182,mao,Mahon,39.8885,4.2658,45,
199,broome,Broome,42.2506,-75.8330,70,
200,ierapetra,Ierapetra,35.0100,25.7353,61,
244,es castell,Es Castell,39.8795,4.2854,45,
339,xianyang,Xianyang,34.3378,108.7026,65,
538,stephenville,Stephenville,32.2207,-98.2023,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [56]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey": geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
remire-montjoly - nearest hotel: Complexe Belova
jamestown - nearest hotel: DoubleTree Jamestown
cabo san lucas - nearest hotel: Comfort Rooms
la passe - nearest hotel: L'Estuaire
mao - nearest hotel: Eurohotel
broome - nearest hotel: No hotel found
ierapetra - nearest hotel: No hotel found
es castell - nearest hotel: Hotel Agamenon
xianyang - nearest hotel: 如家精选酒店(西安沣西港沣国际店)
stephenville - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
42,remire-montjoly,Rémire-Montjoly,4.9167,-52.2667,70,Complexe Belova
83,jamestown,Jamestown,42.0970,-79.2353,69,DoubleTree Jamestown
102,cabo san lucas,Cabo San Lucas,22.8909,-109.9124,66,Comfort Rooms
130,la passe,La Passe,45.5549,-0.8967,47,L'Estuaire
182,mao,Mahon,39.8885,4.2658,45,Eurohotel
199,broome,Broome,42.2506,-75.8330,70,No hotel found
200,ierapetra,Ierapetra,35.0100,25.7353,61,No hotel found
244,es castell,Es Castell,39.8795,4.2854,45,Hotel Agamenon
339,xianyang,Xianyang,34.3378,108.7026,65,如家精选酒店(西安沣西港沣国际店)
538,stephenville,Stephenville,32.2207,-98.2023,65,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [57]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = 'Humidity',
    tiles = "OSM",
    hover_cols=['Humidity', 'Hotel Name', 'Country'],
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "City",
    ylabel = "Latitude",
    xlabel = "Longitude",
)

hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)